# **Introduction**

Students write text here, explaining:
 - What it means to preprocess data and what is a preprocessing pipeline
 - What are the main artefacts that can be found in the TMS-EEG signal and their causes
 - **Optional**: a personal comment or opinion on the problems posed by preprocessing, based on what you heard in class during both theoretical lessons and the first _briefing_



**Ans:**

Preprocessing refers to the process of separating signal from noise in acquired data, transforming raw data to allow for ease of analysis. In the context of EEG, this is the separation of scalp voltage generated by the cerebral cortex in response to stimuli (e.g., TMS pulse) from scalp voltage generated by non-cerebral sources.

A preprocessing pipeline refers to the sequential transformation of data, through multiple steps. 

A current challenge posed by the preprocessing of TMS-EEG data is the lack of a standardized pipeline. Pipeline approaches can range from being conservationist to interventionist, varying on a case-by-case basis. Conservational approaches are more likely to preserve data as was acquired, but researchers run the risk of over-emphasising redundant data patterns; interventionist approaches tend to clean data more aggressively, but that can lead to the loss of information of interest. While the lack of a standardized pipeline may allow researchers to adapt to preprocessing different types of datasets more flexibly, this can reduce result replicability.

In [1]:
from pathlib import Path                            # to build a bridge between Python and the filesystem 

import numpy as np                                  # to perform array operations
import matplotlib                                   # this and the following to draw plots
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')                            # to make plots interactive

import mne                                          # to read and manipulate EEG data

from scripts import utils                           # custom functions to shorten the code in this notebook

# **1. Basic Preprocessing (Assignment 1, Deadline 26/11/2025)**

Students write text here, listing the operations that we categorise as "basic preprocessing".

If you cannot remember what the steps are, look at the code in the cells below and you will recall. 


**Ans:** 
1. Loading data
2. Inspecting data
3. Adjusting data (dropping non-EEG channels and setting channel location)
4. Interpolating pulse artefact
5. High-pass filtering

## **1.1. Loading and Inspecting Data**

**Optional but welcome:** students write here any considerations about this step.


**Ans:**

The code below points Python to the data folder (Path("data")) and instructs it to load the "S02C1_M1" datasets ("subject_and_condition" variable), identify the file of interest (S01C1_M1.vhdr), return the output on screen so the user may ensure the correct files were loaded ("print"), and to read it in BrainVision format (mne.io.read_raw_brainvision). The results are assigned to a variable (eeg_data) where EEG is represented as a channels x times matrix, along with accompanying metadata.


In [2]:
data_dir = Path("data")
subject_and_condition = "S02C1_M1"

file_of_interest = list(data_dir.glob(pattern=f"{subject_and_condition}.vhdr"))
print(file_of_interest, "|", type(file_of_interest), "|", file_of_interest[0])

[WindowsPath('data/S02C1_M1.vhdr')] | <class 'list'> | data\S02C1_M1.vhdr


In [3]:
eeg_data = mne.io.read_raw_brainvision(vhdr_fname=file_of_interest[0])

Extracting parameters from data\S02C1_M1.vhdr...
Setting channel info structure...


## **1.2. Drop Unwated Channels**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).


**Ans:**

The code below allows for the identification of channels not of interest to data analysis by identifying those that do not follow typical EEG channel naming conventions. 

The code instructs Python to inspect all channel names in the dataset (for channel_name in eeg_data.ch_names), identify whether the last character of each channel name (channel_name[-1]) ends in an integer (int), and, if not (except ValueError), whether the last character is the letter "z". If a channel name does not fulfill either condition, Python is instructed to store it under the "channels_to_drop" variable. 

The irrelevant and dropped channel names are then displayed on screen (print). Lastly, Python is instructed to remove the irrelevant channel data from the dataset (eeg_data.drop_channels).


In [4]:
channels_to_drop = []

for channel_name in eeg_data.ch_names:
    try:
        int(channel_name[-1])
    except ValueError:
        if channel_name[-1] != "z": 
            channels_to_drop.append(channel_name)
print(f"Channels to drop: {channels_to_drop}")

eeg_data = eeg_data.drop_channels(ch_names=channels_to_drop)

Channels to drop: ['EOG', 'FDI']


## **1.3. Set Channel Location**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

**Ans:**

Head and sensor digitization is integral in data analysis (e.g., in source reconstruction) and visualization (e.g., in constructig topoplots). However, since the current dataset contains neither, where the EEG channels are located in relation to the subject's scalp is unkown. The current step associates each channel to a set of coordinates (a.k.a. montage) on a 2D template head.

The code instructs Python to use the pre-builtin "easycap-M1" montage as reference to generate a montage object (mne.channels.make_standard_montage). The montage object contains the channel names, each channel's x y and z coordinates, as well as identifies the 3 fiducial points. When run (easycap_m1_mongtage.plot()), the 3D coordinates of each channel are projected onto a 2D scalp template to allow for channel position visualization. Python is also instructed to ignore channels that are not available on the easycap-M1 template (on_missing="ignore") 


In [5]:
easycap_m1_montage = mne.channels.make_standard_montage(kind="easycap-M1")
easycap_m1_montage.plot()
eeg_data.set_montage(montage=easycap_m1_montage,
                     on_missing="ignore")

<RawBrainVision | S02C1_M1.eeg, 70 x 3307600 (661.5 s), ~86 KiB, data not loaded>

**Compulsory:** here, students describe and comment the plot generated by the code below. In particular, I would like to read:

- What each row represents
- What are the $x$ and $y$ of each row, and their units of measurement
- A description of the artefacts you see, and their likely causes


**Ans:**

Each row represent the electrical signal acquired from each EEG electrode channel.

$x$ = Time, in seconds; $y$ = EEG electrode name (and thus the general location on scalp)

Artefacts:
- High frequency signals (e.g. channels FC3, C3, C1): Likely generated by environmental noise
- Large fluctuations at event markers: TMS pulse introduces noise to data
- Large, quick fluctuations most prominent at Fp channels, shrinking as channels move increasingly further from the eyes (e.g., timepoint 123s): Artifacts from eye blinks
- Large fluctuations across all channels at specific timepoints (e.g., timepoint 27.7s): Muscle noise

In [6]:
eeg_data.plot()

Using qt as 2D backend.


**Compulsory:** here, students define event markers and explain why they are important for EEG data analysis

**Optional:** any comments that go beyond a mere definition are welcome. For example, this could be comments about the structure of event arrays and what it tells us about the nature of events


**Ans:**

Event markers provide information on the point at which an event (e.g., TMS pulse) occured. This includes its ordinal number, type, name, time of occurence, duration, and the number of channels involved. Event markers serve as a reference point in which researchers can create epochs when generating TMS-evoked potentials.

The code instructs Python to extract marker information from the EEG data structure (mne.events_from_annotations(raw=eeg_data)) and associate each name to a numerical format that is readable by the program (events_dict). An array is formed, which each row (200 total) corresponding to an event's time of occurence, affected channels, and event type. It also instructs Python to extract data from the second event onwards ([2:]) as these events are likely irrelevant, such as test pulses.

In [7]:
events_from_annotations, events_dict = mne.events_from_annotations(raw=eeg_data)
events_from_annotations = events_from_annotations[2:]

Used Annotations descriptions: ['New Segment/', 'Stimulus/S 54', 'Stimulus/S255']


## **1.4. Interpolate the Pulse Artefact**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).


**Ans:**

This step creates small time windoes (epochs) around the TMS pulses, averages them across the 200 trials, and plots the resulting TEPs for each channel.

The code instructs Python to create an epoch around each TMS pulse (mne.Epochs), as extracted from the raw eeg_data file. The time window begins 1.1s before TMS pulse application (tmin=-1.1) and ends 0.5s after the pulse (tmax=0.5). This is repeated for each event (events=events_from_annotations) of the raw eeg data file, and results are stored in the variable "pre_interpolation_epochs_tep". For each channel, the epochs are averaged across trials to create a TEP (pre_interpolation_epochs.average()).

In [8]:
pre_interpolation_epochs = mne.Epochs(raw=eeg_data,
                                      events=events_from_annotations,
                                      tmin=-1.1,
                                      tmax=0.5,
                                      baseline=None)
pre_interpolation_epochs_tep = pre_interpolation_epochs.average()
pre_interpolation_epochs_tep.plot();

Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Channels marked as bad:
none


In [9]:
post_interpolation_eeg = mne.io.read_raw(fname="data/post_2_5_interpolation_eeg.fif",
                                         preload=True)
post_interpolation_epochs = mne.Epochs(raw=post_interpolation_eeg,
                                       events=events_from_annotations,
                                       tmin=-1.1,
                                       tmax=0.5,
                                       baseline=None)
post_interpolation_tep = post_interpolation_epochs.average()
post_interpolation_tep.plot();

Opening raw data file data/post_2_5_interpolation_eeg.fif...
    Range : 0 ... 3307599 =      0.000 ...   661.520 secs
Ready.
Reading 0 ... 3307599  =      0.000 ...   661.520 secs...
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


## **1.5. High-Pass Filtering**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and potential caveats (that is, one-two things that can happen if you do this step unproperly).


**Ans:**

This step sets up a high-pass filter to supress extremely low frequencies within the signal, potentially caused by artefactual sources such as sweating.

The code applies a high-pass filter (mne.filter.filter_data) to attenuate signals from the data lower than 0.1Hz (1_freq=0.1). No low-pass filter is applied (h_freq=None). The filtered NumPy data is then converted into a raw object (mne.io.RawArray). The new data is epoched again for all channels (mne.Epochs) with the same time window as before (from 1.1s before a TMS pulse to 0.5s after), then averaged to generate the post-filtered TEPs (post_filtering_epochs.average()).

Some potential caveats when applying filters include the accidental suppression of low-oscillatory signals of interest (e.g., alpha rhythm) when an inapporpriate filter threshold is applied. Excessive use of temporal filters may also lead to the distortion of signal amplitude and latency or the introduction of artificial peaks within the signal.

In [12]:
post_filtering_eeg = mne.filter.filter_data(data=post_interpolation_eeg.get_data(),
                                            sfreq=post_interpolation_eeg.info["sfreq"],
                                            l_freq=0.1,
                                            h_freq=None,
                                            method="iir",
                                            iir_params=None,
                                            copy=True,
                                            phase="zero")
post_filtering_eeg = mne.io.RawArray(data=post_filtering_eeg,
                                     info=post_interpolation_eeg.info)
post_filtering_epochs = mne.Epochs(raw=post_filtering_eeg,
                                   events=events_from_annotations,
                                   tmin=-1.1,
                                   tmax=0.5,
                                   baseline=None)
post_filtering_tep = post_filtering_epochs.average()
post_filtering_tep.plot();

Setting up high-pass filter at 0.1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.10 Hz: -6.02 dB

Creating RawArray with float64 data, n_channels=70, n_times=3307600
    Range : 0 ... 3307599 =      0.000 ...   661.520 secs
Ready.
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


# **2. Independent Component Analysis (ICA) (Assignment 2, Deadline 05/12/2025)**

## **2.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on its rationale, devoting particular attention to discussing the independence assumption and why its application to brain data is debatable.


**Ans:**

An Independent Component Analysis (ICA) is a mathematical procedure that breaks down, or decomposes, a signal into a set of factors. These factors must: 
1. Be linearly independent of each other (i.e., no factor can be expressed mathematically by another factor within the signal) and,
2. After deconstruction, be reconstructable into a singular signal.

In EEG preprocessing, ICAs isolate artefacts from cortical sources in a dataset. By breaking down a signal into its individual factors, researchers can identify factors most likely generated through non-neurocognitive activty (e.g., eye blinks, eye movements, or muscle activity), remove these factors, and reassemble the remaining factors to construct a cleaner signal with a lower signal-to-noise ratio for analysis.

That said, ICA is a blind source-separation technique that does not account for the nature of the sources it decomposes. In the context of EEG preprocessing, this means that ICAs operate under the assumption that factors making up an EEG signal are independent from one another, further implying an assumption that neural activity in different regions of the brain do not interact with one another. This is inaccurate, given the high level of connectivity different neural systems share with one another. Additionally, ICAs assume that EEG sources remain spatially stationary throughout the period of signal recording. This does not account for the possibility that brain activity can spread across the scalp over time, such as during sleep or seizures. 

The incompatability between ICA's independence assumption and the brain's high level of interconnectivity means it is possible that components extracted through ICA may contain multiple neural sources that happened to have shared synchronous activity during data recording. Signals arising from the same brain phenomenon that spreads across space might also be split into separate components.  

Ultimately, components extracted through ICA are purely mathematical and may not be necessarily meaningful.



That said, ICAs are used extensively in the removal of artefacts -- eye blinks in particular. This is because eye blinks are relatively independent of neurocognitive activity as they hold little cognitive significance and have a stereotypical pattern and amplitude that are easily distinguishable from brain signals. ICAs may also be used to remove mucle artefacts, but this practise varies from lab to lab and should be used with caution.


## **2.2. Fitting**

In [20]:
ica = mne.preprocessing.ICA(random_state=0)
ica.fit(post_filtering_epochs)

Fitting ICA to data using 70 channels (please be patient, this may take a while)
Using data from preloaded Raw for 200 events and 8001 original time points ...
Selecting by non-zero PCA components: 70 components
Using data from preloaded Raw for 200 events and 8001 original time points ...
Fitting ICA took 375.9s.


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,86 iterations on epochs (1600200 samples)
ICA components,70
Available PCA components,70
Channel types,eeg
ICA components marked for exclusion,—


## **2.3. Components Selection**

**Compulsory:** students select components whose most likely source are eye movements and justify their choices here, including a picture of the selected component(s).

To help you tell components apart, you can refer to [this](https://labeling.ucsd.edu/tutorial/labels) tutorial by the developers of ICLabel: an algorithm to automatically classify independent components that came out of the Swartz Center for Computational Neuroscience, University of California San Diego ([Pion-Tonachini et al., 2019](https://www.sciencedirect.com/science/article/pii/S1053811919304185)).

To include a picture of the selected component(s), just take a screenshot, save it somewhere and change the path below (`files/sample-ic.png`) with the path to your screenshot. 

Selecting no components is OK but the choice must be justified in writing.


**Ans:**

ICA001 is most likely the component whose sources are eye blinks or eye movements. This is because on its scalp topography, the component visibly affects the region around the eyes, and the power spectrum graph shows small and few peaks. When inspecting the component over time, ICA001 shows large, periodic, and steretypical deflections. The component fluctuations also correspond to the periodic, large fluctuations (likely caused by eye blinks) recorded by the Fp1 channel in the raw data. 

ICA021 might also be a product of eye movement, possibly from saccades or horizontal eye movement. This is because the component affects the region around the eyes, is dipolar, and the power specturm graph displays small and few peaks.

<p align="center">
<img src="C:\Users\Todd\OneDrive\Documents\Masters\Brain Stimulation and Multimodal Electrophysiological Recording\brainstim-multimodal-main\ICA001.png" width="1000"/>
</p>

<p align="center">
<img src="C:\Users\Todd\OneDrive\Documents\Masters\Brain Stimulation and Multimodal Electrophysiological Recording\brainstim-multimodal-main\ICA001 Time.png" width="1000"/>
</p>

<p align="center">
<img src="C:\Users\Todd\OneDrive\Documents\Masters\Brain Stimulation and Multimodal Electrophysiological Recording\brainstim-multimodal-main\ICA021.png" width="1000"/>
</p>

In [21]:
ica.plot_components(inst=post_filtering_epochs, picks=range(30))

<MNEFigure size 975x1170 with 30 Axes>

In [22]:
ica.plot_sources(inst=post_filtering_epochs)

Using data from preloaded Raw for 200 events and 8001 original time points ...
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated


In [23]:
components_to_reject = utils.select_items(item_type="ica")

In [24]:
ica.exclude = components_to_reject
ica.apply(post_filtering_epochs.load_data())
post_ica_epochs = post_filtering_epochs
post_ica_tep = post_ica_epochs.average()
post_ica_tep.plot();

Using data from preloaded Raw for 200 events and 8001 original time points ...
Applying ICA to Epochs instance
    Transforming to ICA space (70 components)
    Zeroing out 1 ICA component
    Projecting back using 70 PCA components


# **3. Manual Artifact Rejection (Assignment 3, Deadline 12/12/2025)**

## **3.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on the criteria to follow for its execution. 

## **3.2. Execution**

In [ ]:
post_ica_epochs.plot()

# **4. Computing & Assessing a TEP (Assignment 4, Deadline 19/12/2025)**

## **4.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on the criteria to follow for its execution. 

## **4.2. Execution**

**Optional:** Student can write here any comment or consideration that goes beyond the minimum required, for example:
- "I think filtering is `{good/bad}` because..."
- "I think these data were particularly `{dirty/clean}` because..."
- Anything else you might feel like saying about the preprocessing you have done

Do not be afraid to make such comments: there is no right or wrong and they do not count for the vote. At most, a great comment could make you a _cum laude_ candidate, but I really just want to see if you have a personal opinion on the subject.